# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [52]:

import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [53]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [54]:
from hidden import IEX_CLOUD_API_TOKEN


## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [55]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 56647584, 'calculationPrice': 'tops', 'change': 0.07, 'changePercent': 0.00041, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': None, 'highTime': None, 'iexAskPrice': 169.96, 'iexAskSize': 300, 'iexBidPrice': 167.7, 'iexBidSize': 101, 'iexClose': 169.96, 'iexCloseTime': 1714149372344, 'iexLastUpdated': 1714149372344, 'iexMarketPercent': 0.01382165668044919, 'iexOpen': 169.86, 'iexOpenTime': 1714138200024, 'iexRealtimePrice': 169.96, 'iexRealtimeSize': 10, 'iexVolume': 265719, 'lastTradeTime': 1714149372344, 'latestPrice': 169.96, 'latestSource': 'IEX real time price', 'latestTime': '12:36:12 PM', 'latestUpdate': 1714149372344, 'latestVolume': None, 'low': None, 'lowSource': None, 'lowTime': None, 'marketCap': 2624502094760, 'oddLo

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [56]:
price = data['latestPrice']
market_cap = data['marketCap']
print(market_cap)

2624502094760


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [57]:
my_columns = ['Ticker','Stock Price','Market Capitalization','Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [58]:
final_dataframe._append(
    pd.Series([
        symbol,
        price,
        market_cap,
        'N/A'
        ],index=my_columns
    ),ignore_index=True
)


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,169.96,2624502094760,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://cloud.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    try:
        data = requests.get(api_url).json()
        final_dataframe = final_dataframe._append(
        pd.Series([
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
            ],index=my_columns),
            ignore_index=True
    )
        
    except Exception as e:  # Catch any exception (including JSONDecodeError)
        print(f"Error getting data for {stock}: {e}")
        continue
        

Error getting data for ABC: Expecting value: line 1 column 1 (char 0)
Error getting data for ANTM: Expecting value: line 1 column 1 (char 0)
Error getting data for COG: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error getting data for DISCA: Expecting value: line 1 column 1 (char 0)
Error getting data for EMR: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error getting data for FBHS: Expecting value: line 1 column 1 (char 0)
Error getting data for HFC: Expecting value: line 1 column 1 (char 0)
Error getting data for NLOK: Expecting value: line 1 column 1 (char 0)
Error getting data for RE: Expecting value: line 1 column 1 (char 0)
Error getting data for VIAC: Expecting value: line 1 column 1 (char 0)
Error getting data for WLTW: Expecting value: line 1 column 1 (char 0)


In [9]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,136.37,39963949079,N/A
1,AAL,14.13,9251713810,N/A
2,AAP,74.71,4453423578,N/A
3,AAPL,169.89,2623421163090,N/A
4,ABBV,167.29,296211533786,N/A
...,...,...,...,...
489,YUM,141.56,39849140000,N/A
490,ZBH,119.75,24604008757,N/A
491,ZBRA,292.53,15035319743,N/A
492,ZION,41.27,6091411555,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [59]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [60]:

symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
  #  print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            latest_price = data[symbol]['quote']['latestPrice']
            market_cap = data[symbol]['quote']['marketCap']
            final_dataframe = final_dataframe._append(
        pd.Series([symbol, latest_price, market_cap, 'N/A'],
                   index=my_columns),
        ignore_index=True)
        except KeyError:
            #latest_price = 'N/A'
            #market_cap = 'N/A'
            print(f"Symbol '{symbol}' not found in data")
final_dataframe

Symbol 'ABC' not found in data
Symbol 'ANTM' not found in data
Symbol 'DISCA' not found in data
Symbol 'FBHS' not found in data
Symbol 'HFC' not found in data
Symbol 'NLOK' not found in data
Symbol 'RE' not found in data
Symbol 'VIAC' not found in data
Symbol 'WLTW' not found in data


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,137.895,40410858387,N/A
1,AAL,13.705,8973442163,N/A
2,AAP,75.180,4481440030,N/A
3,AAPL,169.970,2624656513570,N/A
4,ABBV,159.525,282462459963,N/A
...,...,...,...,...
491,YUM,142.470,40105305000,N/A
492,ZBH,119.430,24538261093,N/A
493,ZBRA,297.340,15282541868,N/A
494,ZION,42.070,6209490771,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [61]:
portfolio_size = input('Enter value of portfolio: ')
try:
    val = float(portfolio_size)
       
except ValueError:
    print('This is not a number \nPlease enter a number')
    input('Enter value of portfolio: ')
    val = float(portfolio_size)

Enter value of portfolio: 1000000


In [62]:
position_size = val/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Stock Price'])
    
    
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,137.895,40410858387,14
1,AAL,13.705,8973442163,147
2,AAP,75.180,4481440030,26
3,AAPL,169.970,2624656513570,11
4,ABBV,159.525,282462459963,12
...,...,...,...,...
491,YUM,142.470,40105305000,14
492,ZBH,119.430,24538261093,16
493,ZBRA,297.340,15282541868,6
494,ZION,42.070,6209490771,47


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [79]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [80]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [81]:
#writer.sheets['Recommended Trades'].set_column('A:A',18,string_format)
#writer.sheets['Recommended Trades'].set_column('B:B',18,string_format)
#writer.sheets['Recommended Trades'].set_column('C:C',18,string_format)
#writer.sheets['Recommended Trades'].set_column('D:D',18,string_format)


This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [82]:

column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)
    writer.close()

## Saving Our Excel Output

Saving our Excel file is very easy: